### How to represent words for NLP
* motivator on why bag of words/ one hot encoding is bad
    * curse of dimensionality, sparsity, ignores context, new words, etc.
* Word vectors
    * distributional hypothesis 
        * describing the landscape of models as using different types of "context"   
        * count and predictive approachs [<a href="#note1">note.</a>]
        * larger context: semantic relatedness (e.g. “boat” – “water”)
        * smaller context: semantic similarity (e.g. “boat” – “ship”)
    * quick overview on methods
    * SVD on doc/word matrices
    * SVD on co-occurance matrices with window
    
    * some issues:
        * large matrices!
        * expensive to SVD (quadratic time)
        * Sparse
    * Glove
    * word2vec: make word vectors the parameters of a model with the objective of defining local context.
    * go over word2vec in a little more detail
        * skip gram
        * cbow
        * negative sampling
    * word embeddings in python:
        * sklearn/pydsm + numpy (vectorizers + matrix decompositions)
        * gensim (word2vec)
* Neural models 

        
        
* Inspecting results of word embeddings:
    * self organizing maps
* Validating word vectors:
    * intrinsic vs extrinsic
    
* A note on NNs:
    * transferable features in shallow parts of a network, theres an analogy their with word2vec (shallow networks).

### Exercise 1:
Train your own word2vec model using dataset, and load those vectors into spacy. Visually inspect the results of the vector as a self organizing map.

In [2]:
!pip install gensim >> gensim-log.txt
from gensim.models import Word2Vec
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups()
corpus = dataset.data







In [12]:
#!pip install spacy >> spacy-install.log
#!python -m spacy download en >> spacy-download.log
import spacy
nlp = spacy.load('en')

In [13]:
doc = nlp(corpus[0])

In [ ]:
from spacy.tokens import Doc
import spacy
from spacy.matcher import Matcher
from spacy.attrs import ORTH, IS_PUNCT

def merge_phrases(matcher, doc, i, matches):
    '''
    Merge a phrase. We have to be careful here because we'll change the token indices.
    To avoid problems, merge all the phrases once we're called on the last match.
    '''
    if i != len(matches)-1:
        return None
    # Get Span objects
    spans = [(ent_id, label, doc[start : end]) for ent_id, label, start, end in matches]
    for ent_id, label, span in spans:
        span.merge(label=label, tag='NNP' if label else span.root.tag_)

def process_token(token):
    if token.is_punct or token.is_space:
        return False
    elif token.like_url:
        return'URL'
    elif token.like_email:
        return'EMAIL'
    elif token.like_num:
        return'NUM'
    else:
        return token.lower_.replace(" ",'')
    
    
def process_sentence(tokenized_sent):
    tokens = []
    
    doc = Doc(nlp.vocab, words = tokenized_sent)
    nlp.tagger(doc)
    matcher(doc)
    
    for token in doc:
        processed_token = process_token(token)
        if processed_token:
            tokens.append(processed_token)    
    return tokens



In [102]:
from spacy.tokens import Doc
import spacy
from spacy.matcher import Matcher
from spacy.attrs import ORTH, IS_PUNCT

class TextProcesser(object):
    def __init__(self, nlp=None):
        self.nlp = nlp or spacy.load('en')
        
    def __call__(self, corpus):
        for doc in self.nlp.pipe(corpus, parse=False):
            for ent in doc.ents:
                ent.merge()
            yield from map(self.process_token, doc)

            
    def process_token(self, token):
        if token.like_url:
            return'URL'
        elif token.like_email:
            return'EMAIL'
        elif token.like_num:
            return'NUM'
        else:
            return token.lower_

In [103]:
T = TextProcesser(nlp)

In [104]:
t = T(corpus)

In [107]:
g = list(t)

In [ ]:
model = Word2Vec(sentences=processed_sents, ###tokenized senteces, list of list of strings
                 size=300,  #size of embedding vectors
                 workers=8, #how many threads?
                 min_count=5, #minimum number of token instances to be considered
                 sample=0, #weight of downsampling common words? 
                 sg = 0, #should we use skip-gram? if 0, then cbow
                 hs=0, #heirarchical softmax?
                 iter=5 #training epocs
        )

In [ ]:
<p id="note1">
turns out the distinction may not be that important. (see [Levy and Goldberg (2014), Pennington et al. (2014), Österlund et al. (2015)] as referenced in https://www.gavagai.se/blog/2015/09/30/a-brief-history-of-word-embeddings/).
</p>

### Tensorflow model outline
#### Definitions phrase
* Decide on an architecture
* Define all variables as tensors
* Define how to generate outputs from your inputs and variables
* Define a cost function with respect to your predictions and you labels
* Define an optimizer that minimizes your cost function
#### Execution phase
* create an execution session
* Initialize your variables
* over n epochs, run the optimizer, feeding it some data in batches

In [25]:
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import label_binarize
import numpy as np
cancer = load_breast_cancer()
X = cancer.data
y = label_binarize(cancer.target, classes=[0,1,2])[:, :2]


In [17]:
def init_vector(shape):
    init_vals = tf.random_normal(shape)
    return tf.Variable(init_vals)

def feed_forward(X, weights):
    hidden = tf.nn.relu(tf.matmul(X, weights['weights1']) + weights['bias1'])
    output = tf.matmul(hidden, weights['weights2']) + weights['bias2']
    return tf.nn.softmax(tf.nn.sigmoid(output))


In [15]:
input_rows = X.shape[0]
input_dim = X.shape[1]
hidden_1_dim = 100
n_classes = len(np.unique(y))

weights = {
    'weights1':init_vector((input_dim,hidden_1_dim )), 
    'bias1':init_vector((hidden_1_dim, ))
    'weights2':init_vector((hidden_1_dim,n_classes )), 
    'bias2':init_vector((n_classes, ))    
}



cost = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=predictions)

In [27]:
tf.nn.softmax_cross_entropy_with_logits?